 <h1 style="text-align:center"> Irish energy data preparation  </h1>
 
## Outline

In this notebook I take data available from the Sustainable Energy Authority of Ireland (SEAI) including primary energy from 1990 to 2022 and final energy from 1990 to 2022. 

I merge the data to be able to compare the two over the time series.

## Data sources

Primary energy and final consumption (ktoe) 1990 - 2022 data was obtained from the SEAI energy portal:

https://www.seai.ie/data-and-insights/seai-statistics/energy-data/

Primary Energy Production (ktoe) with fuel type 1990 - 2022 was obtained from the central statistics office portal:

https://data.cso.ie

## Merging final energy and final consumption dataframes

In order to look at trends in primary energy over time and compare this with final consumption the two dataframes were merged using 'Year' and fuel type.

In [1]:
# import python libraries

import pandas as pd # pandas for panel data
import plotly.express as px # plotly for interactive plots
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# load the data

df_primary_energy = pd.read_csv("../data/Historic_ktoe_by_Fuel.csv")
df_final_energy = pd.read_csv("../data/Historic_ktoe_by_Fuel_consumption.csv")

# total consumption data has fuel groups broken up in to smaller groups e.g type coal. 
# Want to collapse this so can merge with primary energy

df_final_energy = df_final_energy.groupby(
    ["Year", "Fuel Group Hierarchy - Fuel Group"])["Final Consumption"].sum().reset_index()

# merge primary and final energy dataframes using 'Year' and 'Fuel Group' columns
total_energy = pd.merge(df_primary_energy, df_final_energy, on=['Year','Fuel Group Hierarchy - Fuel Group'])

# rename Fuel Group for abbreviated header
total_energy.rename(columns={'Fuel Group Hierarchy - Fuel Group': 'Fuel Group'}, inplace=True)


In [4]:
# write this df to the data folder

total_energy.to_csv("../data/Irish_energy_summary1990_2022.csv", sep = ",", header=True, index=False)

## Filtering primary energy source by renewables with subgroups of renewables

In [47]:
# load the data

df_renewables = pd.read_csv("../data/irish_energy_renewables1990_2022.csv")

In [48]:
# do not need the statistic label for plotting
df_renewables.drop(columns="Statistic Label", inplace=True)

# only keep renewable energy here
columns_keep = ['Hydro', 'Wind', 'Biomass', 'Landfill gas', 'Biogas', 'Liquid Biofuel', 'Solar', 'Geothermal']

df_renewables_filtered = df_renewables[df_renewables["Fuel Type"].isin(columns_keep)]

# change VALUE to integer, will coerce "-" in to null value
df_renewables_filtered['VALUE'] = pd.to_numeric(df_renewables_filtered['VALUE'], errors='coerce').astype('Int64')

/var/folders/7f/dffk3cfs1gzfgm16g6wd2_2w0000gn/T/ipykernel_3037/3173708438.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [51]:
# write this df to the data folder

df_renewables_filtered.to_csv("../data/Irish_renewable_energy_summary1990_2022.csv", sep = ",", header=True, index=False)

# Electricity generated by fuel type 

In [33]:
# load the df, values are in ktoe

df_electricity = pd.read_csv("../data/Elec-generated-by-fuel.csv")

In [34]:
df_electricity.rename(columns={"Electricity generated by fuel type; ktoe":"Year"}, inplace=True)

In [35]:
# list of non-renewables to sum up
list_non_renewables = ['Gas', 'Coal', 'Peat', 'Oil', 'Wastes Non-Renewable', 'Net Positive Electricity Imports']

# list of all to sum up, except year
list_all = df_electricity.columns.drop('Year')

# create new column for all non-renewable, and all types
df_electricity['Non-renewable sum (ktoe)'] = df_electricity[list_non_renewables].sum(axis=1) 
df_electricity['Sum energy sources (ktoe)'] = df_electricity[list_all].sum(axis=1)

In [36]:
# write this to csv
df_electricity.to_csv("../data/Irish_electricity_generation_sources.csv", sep = ",", header=True, index=False)

In [38]:
df_electricity.columns

Index(['Year', 'Gas', 'Coal', 'Peat', 'Oil', 'Wastes Non-Renewable',
       'Renewables', 'Net Positive Electricity Imports',
       'Non-renewable sum (ktoe)', 'Sum energy sources (ktoe)'],
      dtype='object')